# Instance_Segmentation_Pytorch (Inference)

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
1. [定義模型方法](#5)
1. [製作資料集＆資料擴增＆推論模型](#6)
1. [待辦事項](#7)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
# 資料處理套件
import os
import gc
import cv2
import time
import random
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# pytorch深度學習模組套件
import torch
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as F
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
# 查看pytorch版本
print(torch.__version__)

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'sartorius-cell-instance-segmentation/' 

# (String)CSV根路徑
CSV_ROOT_PATH = PATH+r'sartorius-cell-instance-segmentation/'

# (String)測試資料路徑
TEST_DATA_PATH = DATA_ROOT_PATH+r'test/'

# (String)測試CSV路徑
TEST_CSV_PATH = CSV_ROOT_PATH+r'sample_submission.csv'

# (Boolean)是否要匯入Library
IMPORT_PYTORCH_LIBRARY = False

# (String)Library的路徑
PYTORCH_LIBRARY_PATH = PATH + "Util/"

# (String)讀取預訓練模型/權重的名稱，當fold model時，後面會自動加_NUMBER
LOAD_MODEL_NAME = ['maskrcnn_resnet50_fpn']

# (String)讀取預訓練模型/權重的儲存路徑
LOAD_MODEL_PATH = [PATH + r'test1/']

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
# Override pythorch checkpoint with an "offline" version of the file
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/pytorch-pretrained-models/resnet50-19c8e357.pth /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Dict)標籤字典
LABEL_DICT = {"astro": 1, "cort": 2, "shsy5y": 3}

# (Int)集成模型數量
ENSEMBLE_MODEL_COUNT = 1

# (Int List)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = [1]*ENSEMBLE_MODEL_COUNT

# (Int)圖片尺寸寬
IMAGE_SIZE_W = [704]*ENSEMBLE_MODEL_COUNT

# (Int)圖片尺寸高
IMAGE_SIZE_H = [520]*ENSEMBLE_MODEL_COUNT

# (String)CSV圖片檔名欄位
IMAGE_NAME = "id"

# (String)CSV標注預測欄位
ANNOTATION_NAME = "predicted"

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = False

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.png'

# (Flag)圖像讀取的格式
IMREAD_FLAGS = cv2.IMREAD_COLOR

#  (Boolean)圖像是否要轉換
COLOR_CONVERT = False
if COLOR_CONVERT:
    #  (Boolean)圖像轉換通道
    COLOR_CONVERT_CHANNEL = cv2.COLOR_BGR2RGB

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

# (Boolean)如為True每次返回的卷積算法將是確定的，即默認算法
CUDNN_DETERMINISTIC = True

# (Boolean)PyTorch 中對模型裡的卷積層進行預先的優化，也就是在每一個卷積層中測試 cuDNN 提供的所有卷積實現算法，
# 然後選擇最快的那個。這樣在模型啟動的時候，只要額外多花一點點預處理時間，就可以較大幅度地減少訓練時間
CUDNN_BENCHMARK = True


'''資料擴增參數設定'''

# (Boolean)是否圖形歸一化
IS_NORMALIZE = False

# (Tuple Float)正規化的平均值((0,1)的參考平均值:(0.485, 0.456, 0.406), (-1,1)的參考平均值:(0.5, 0.5, 0.5)
NORMALIZE_MEAN = (0.485, 0.456, 0.406)

# (Tuple Float)正規化的標準差((0,1)的參考標準差(0.229, 0.224, 0.225), (-1,1)的參考標準差(0.5, 0.5, 0.5)
NORMALIZE_STD = (0.229, 0.224, 0.225)

# (Float)水平翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HORIZONTALFLIP = 0.5

# (Float)垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VERTICALFLIP = 0.5


''''模型參數設定'''

# (String List)模型載入方式 - 1 MODEL;2 WEIGHT_OF_CUSTOM_MODEL;
# 3 WEIGHT_OF_BASE_MODEL
MODEL_LIST = [3] * ENSEMBLE_MODEL_COUNT

if 2 in MODEL_LIST:
    # (Model List)模型載入方式有CUSTOM_MODEL，依照index位置填入
    CUSTOM_MODEL = [None] * ENSEMBLE_MODEL_COUNT

if 3 in MODEL_LIST:
    # (Model List)模型載入方式有BASE_MODEL，依照index位置填入
    BASE_MODEL = [torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained = False, box_detections_per_img = 540)] * ENSEMBLE_MODEL_COUNT

    # (Int)模型隱藏層
    HIDDEN_LAYER = 256

    # (String)模型Box預測
    MODEL_BOX_PREDICTOR = FastRCNNPredictor

    # (String)模型Mask預測
    MODEL_MASK_PREDICTOR = MaskRCNNPredictor

# (Boolean)是否印出完整模型
MODEL_PRINT = False


''''推論參數設定'''

# (Int List)每批推論的尺寸
BATCH_SIZE = [2]*ENSEMBLE_MODEL_COUNT

# (Int)指定列印進度條的位置（從0開始）
TQDM_POSITION = 0

# (Boolean)保留迭代結束時進度條的所有痕跡。如果是None，只會在position是0時離開
TQDM_LEAVE = True


''''評價指標參數設定'''

# (Float dist)標籤分類的最小允許分數，否則不評價
MIN_SCORE_DICT = {1: 0.55, 2: 0.75, 3: 0.5}

# (Float dist)Mask的最小允許閾值，否則不評價
MASK_THRESHOLD_DICT = {1: 0.55, 2: 0.75, 3:  0.6}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = CUDNN_DETERMINISTIC
    torch.backends.cudnn.benchmark = CUDNN_BENCHMARK

seed_everything(SEED)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
print('Reading data...')

# 讀取訓練資料集CSV檔
if os.path.isfile(TEST_CSV_PATH):
    test_csv = pd.read_csv(TEST_CSV_PATH,encoding="utf8")
else:
    test_data_directory_list = os.listdir(TEST_CSV_PATH)
    test_csv = pd.DataFrame(test_data_directory_list, columns=[IMAGE_NAME])
    del test_data_directory_list
    gc.collect()

print('Reading data completed')

In [ ]:
# 顯示測試資料集CSV檔
test_csv.head()

In [ ]:
print("Shape of train_data :", test_csv.shape)

# 5. 定義模型方法<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

In [ ]:
def build_model(count, model_path):
    if MODEL_LIST[count] == 1:
        # 載入預訓練模型
        model = torch.load(model_path)
    else:
        if MODEL_LIST[count] == 2:
            # 載入模型架構
            model = CUSTOM_MODEL[count]
        elif MODEL_LIST[count] == 3:
            model = BASE_MODEL[count]

            # get the number of input features for the classifier
            in_features = model.roi_heads.box_predictor.cls_score.in_features
            # replace the pre-trained head with a new one
            model.roi_heads.box_predictor = MODEL_BOX_PREDICTOR(in_features, len(LABEL_DICT)+1)

            # now get the number of input features for the mask classifier
            in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
            # and replace the mask predictor with a new one
            model.roi_heads.mask_predictor = MODEL_MASK_PREDICTOR(in_features_mask, HIDDEN_LAYER, len(LABEL_DICT)+1)
        
    if MODEL_LIST[count] != 1:
        # 載入預訓練權重
        model.load_state_dict(torch.load(model_path))
        
    for param in model.parameters():
        param.requires_grad = False
        
    return model

# 6. 製作資料集＆資料擴增＆推論模型<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

In [ ]:
def rle_encoding(x: np.ndarray):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))


def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

In [ ]:
class MyDataset(Dataset):
    def __init__(self, transforms = None):
        self.transforms = transforms
        self.image_ids = [f[:-4]for f in os.listdir(TEST_DATA_PATH)]
            
    def __len__(self):
        return len(self.image_ids)
            
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        image_path = os.path.join(TEST_DATA_PATH, image_id + IMAGE_NAME_EXTENSION)
        image = cv2.imread(image_path, IMREAD_FLAGS)
        image = cv2.resize(image, (IMAGE_SIZE_W[0], IMAGE_SIZE_H[0]))
        
        if COLOR_CONVERT:
            image = cv2.cvtColor(image, COLOR_CONVERT_CHANNEL)

        if self.transforms:
            image, _ = self.transforms(image = image, target = None)
            
        return {'image': image, 'image_id': image_id}

In [ ]:
# These are slight redefinitions of torch.transformation classes
# The difference is that they handle the target and the mask
# Copied from Abishek, added new ones
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class VerticalFlip:
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-2)
            bbox = target["boxes"]
            bbox[:, [1, 3]] = height - bbox[:, [3, 1]]
            target["boxes"] = bbox
            target["masks"] = target["masks"].flip(-2)
        return image, target

class HorizontalFlip:
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-1)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
            target["masks"] = target["masks"].flip(-1)
        return image, target

class Normalize:
    def __call__(self, image, target):
        image = F.normalize(image, NORMALIZE_MEAN, NORMALIZE_STD)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

def get_transforms():
    transforms = [ToTensor()]
    if IS_NORMALIZE:
        transforms.append(Normalize())

    return Compose(transforms)

In [ ]:
def inference_one_epoch(model, test_dataloader, 
                        min_score_dict: dict={1: 0.55, 2: 0.75, 3: 0.5}, 
                        mask_threshold_dict: dict={1: 0.55, 2: 0.75, 3:  0.6}):
    model.eval()
    pbar = tqdm(enumerate(test_dataloader), total=len(test_dataloader), 
                position = TQDM_POSITION, leave = TQDM_LEAVE)
    for batch_idx, (sample) in pbar:
        image = sample['image']
        image_id = sample['image_id']
        with torch.no_grad():
            result = model([image.to(DEVICE)])[0]

        previous_masks = []
        for i, mask in enumerate(result["masks"]):

            # Filter-out low-scoring results.
            score = result["scores"][i].cpu().item()
            label = result["labels"][i].cpu().item()
            if score > min_score_dict[label]:
                mask = mask.cpu().numpy()
                # Keep only highly likely pixels
                binary_mask = mask > mask_threshold_dict[label]
                binary_mask = remove_overlapping_pixels(binary_mask, previous_masks)
                previous_masks.append(binary_mask)
                rle = rle_encoding(binary_mask)
                submission.append((image_id, rle))
                
        # Add empty prediction if no RLE was generated for this image
        all_images_ids = [image_id for image_id, rle in submission]
        if image_id not in all_images_ids:
            submission.append((image_id, ""))

In [ ]:
def inference_process(count, fold, kf, submission):
    if kf:
        print('Model %i : Fold %i - image size W:%i H:%i with %s and batch size %i'%(count+1, fold, IMAGE_SIZE_W[count], IMAGE_SIZE_H[count], LOAD_MODEL_NAME[count].upper(), BATCH_SIZE[count]))
    else:
        print('Model %i : Image size W:%i H:%i with %s and batch_size %i'%(count+1, IMAGE_SIZE_W[count], IMAGE_SIZE_H[count], LOAD_MODEL_NAME[count].upper(), BATCH_SIZE[count]))
    
    test_dataset = MyDataset(transforms = get_transforms())

    if kf:
        model_path = LOAD_MODEL_PATH[count] + LOAD_MODEL_NAME[count] + '_' + str(fold) + '.pth'
    else:
        model_path = LOAD_MODEL_PATH[count] + LOAD_MODEL_NAME[count] + '.pth'

    model = build_model(count, model_path)
    model = model.to(DEVICE)
    
    inference_one_epoch(model, test_dataset, MIN_SCORE_DICT, MASK_THRESHOLD_DICT)
    
    df_sub = pd.DataFrame(submission, columns=[IMAGE_NAME, ANNOTATION_NAME])
    if kf:
        df_sub.to_csv("submission_model" + str(count+1) + "_fold" + str(fold) + ".csv", index=False)
    elif ENSEMBLE_MODEL_COUNT != 1 and not kf:
        df_sub.to_csv("submission_model" + str(count+1) + ".csv", index=False)
    else:
        df_sub.to_csv("submission.csv", index=False)
    print(df_sub.head())
    
    del test_dataset, model
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
submission = []

In [ ]:
def main():
    try:
        print('Inference start')
        since = time.time()
        for count in range(len(LOAD_MODEL_NAME)):
            if FOLD[count] > 1:
                for fold in enumerate(np.arange(FOLD[count]), 1):
                    inference_process(count, fold = fold, kf = True, submission = submission)
            else:
                inference_process(count, fold = 0, kf = False, submission = submission)
        time_elapsed = time.time() - since
        print('Inference complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    except Exception as exception:
        print(exception)
        raise

In [ ]:
if __name__ == '__main__':
    main()

# 7. 待辦事項<a class="anchor" id="7"></a>
[Back to Table of Contents](#0)